### Self, experiment #1  
From g_devel/b_train-on-one.ipynb  
Decode on the same cohort as training, exprmnt 1/4  
Starting with miR, should also work on GEXP, this could be the demo version  
    miR i/o: g_devel/data_miR_col_norm / g_devel/self-decoded_miR - goal is to write decoded miR files then merge with the origial in an overlay plot  
    GEXP i/o:
    
input GEXP written from: a_GEXP_5k_col_norm.ipynb
  
The input files are from a_step_up_miR.ipynb, this can renamed a_miR_demo  
Original intersection check is in a_ntrsct_01.ipynb, although this is not necassary because the 23 cohorts with miR all have the same miR features  

#### Original notes, from g_devel/b_train-on-one.ipynb  
demo version, re-run on data_5k_stacked  

e_ classifier template ?  

want to convert to single stacked object file read  

From version 01, deleting train-of-though devel notes in this v2  

In [46]:
%whos

Variable                    Type                Data/Info
---------------------------------------------------------
Activation                  type                <class 'keras.layers.core.activation.Activation'>
BatchNormalization          type                <class 'keras.layers.norm<...>tion.BatchNormalization'>
Callback                    type                <class 'keras.callbacks.Callback'>
CustomVariationalLayer      type                <class '__main__.CustomVariationalLayer'>
GEXP_paths                  list                n=25
K                           TFModuleWrapper     <module 'tensorflow.compa<...>ras/backend/__init__.py'>
Layer                       type                <class 'keras.engine.base_layer.Layer'>
Model                       type                <class 'keras.engine.training.Model'>
WarmUpCallback              type                <class '__main__.WarmUpCallback'>
adam                        Adam                <keras.optimizers.optimiz<...>am object at 0x2dc

#### Imports

In [21]:
import time # for fitting run time comparisons

# from sklearn.ensemble import RandomForestClassifier # DT?, send to e_
# from sklearn.metrics import f1_score
# from sklearn.model_selection import train_test_split

# from statistics import mean # sent to e_
# from statistics import stdev

import glob as glob

import pandas as pd

import matplotlib.pyplot as plt # 2022-11-14 move to d_, No, need here for loss plot
# from matplotlib.pyplot import figure # ?

import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import BatchNormalization, Activation, Layer
from tensorflow.keras import metrics, optimizers
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import Callback

import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

# import umap.umap_ as umap # to d_, do for miR?

In [ ]:
# couple potential d_ plot templates

In [47]:
plt_v=17 # miR / GEXP indv. norm self decoding

In [9]:
pwd

'/Users/karlberb/Documents/work/sample_count/synthetic_TCGA_data_gen/g_devel'

In [11]:
miR_paths = sorted(glob.glob('data_miR_col_norm/*.tsv'))

In [33]:
paths = sorted(glob.glob('data_GEXP_5k_col_norm/*.tsv'))

In [12]:
miR_paths

['data_miR_col_norm/ACCmir_col_norm_743.tsv',
 'data_miR_col_norm/BLCAmir_col_norm_743.tsv',
 'data_miR_col_norm/BRCAmir_col_norm_743.tsv',
 'data_miR_col_norm/CESCmir_col_norm_743.tsv',
 'data_miR_col_norm/COADREADmir_col_norm_743.tsv',
 'data_miR_col_norm/ESCCmir_col_norm_743.tsv',
 'data_miR_col_norm/GEAmir_col_norm_743.tsv',
 'data_miR_col_norm/HNSCmir_col_norm_743.tsv',
 'data_miR_col_norm/KIRPmir_col_norm_743.tsv',
 'data_miR_col_norm/LUADmir_col_norm_743.tsv',
 'data_miR_col_norm/LUSCmir_col_norm_743.tsv',
 'data_miR_col_norm/MESOmir_col_norm_743.tsv',
 'data_miR_col_norm/OVmir_col_norm_743.tsv',
 'data_miR_col_norm/PAADmir_col_norm_743.tsv',
 'data_miR_col_norm/PCPGmir_col_norm_743.tsv',
 'data_miR_col_norm/PRADmir_col_norm_743.tsv',
 'data_miR_col_norm/SARCmir_col_norm_743.tsv',
 'data_miR_col_norm/SKCMmir_col_norm_743.tsv',
 'data_miR_col_norm/TGCTmir_col_norm_743.tsv',
 'data_miR_col_norm/THCAmir_col_norm_743.tsv',
 'data_miR_col_norm/THYMmir_col_norm_743.tsv',
 'data_miR_co

In [13]:
len(miR_paths)

23

In [34]:
len(GEXP_paths)

25

### Build Model

In [2]:
# Function set for leave one out loop
def compute_latent(x): # x:
    mu, sigma = x
    batch = K.shape(mu)[0]
    dim = K.shape(mu)[1]
    eps = K.random_normal(shape=(batch,dim), mean=0., stddev=1.0 )
    return mu + K.exp(sigma/2)*eps

In [5]:
# Set custom layer , will need all this at start of shell script
class CustomVariationalLayer(Layer):
    """
    Define a custom layer
    """
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

In [6]:
class WarmUpCallback(Callback):
    def __init__(self, beta, kappa):
        self.beta = beta
        self.kappa = kappa

    def on_epoch_end(self, epoch, logs={}):
        if K.get_value(self.beta) <= 1:
            K.set_value(self.beta, K.get_value(self.beta) + self.kappa)

In [25]:
features

Index(['N:MIR::hsa-let-7a-2-3p:Corrected:', 'N:MIR::hsa-let-7a-3p:Corrected:',
       'N:MIR::hsa-let-7a-5p:Corrected:', 'N:MIR::hsa-let-7b-3p:Corrected:',
       'N:MIR::hsa-let-7b-5p:Corrected:', 'N:MIR::hsa-let-7c-3p:Corrected:',
       'N:MIR::hsa-let-7c-5p:Corrected:', 'N:MIR::hsa-let-7d-3p:Corrected:',
       'N:MIR::hsa-let-7d-5p:Corrected:', 'N:MIR::hsa-let-7e-3p:Corrected:',
       ...
       'N:MIR::hsa-miR-527:Uncorrected:',
       'N:MIR::hsa-miR-548x-3p:Uncorrected:',
       'N:MIR::hsa-miR-5584-5p:Uncorrected:',
       'N:MIR::hsa-miR-670-3p:Uncorrected:',
       'N:MIR::hsa-miR-885-3p:Uncorrected:',
       'N:MIR::hsa-miR-888-5p:Uncorrected:', 'N:MIR::hsa-miR-890:Uncorrected:',
       'N:MIR::hsa-miR-891b:Uncorrected:', 'N:MIR::hsa-miR-892b:Uncorrected:',
       'N:MIR::hsa-miR-892c-3p:Uncorrected:'],
      dtype='object', length=743)

In [ ]:
# miR is giving nans, combine with gexp?

In [ ]:
# running gexp interactive side-by-side with miR

In [ ]:
# index with name added back

In [ ]:
paths = sorted(glob.glob('data_GEXP_5k_col_norm/*.tsv'))

In [41]:
paths = sorted(glob.glob('data_GEXP_5k_col_norm/*.tsv'))
for path in paths:
    frame = pd.read_csv(path, sep = '\t', index_col = 0)
    break

In [42]:
frame

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-OR-A5J1,ACC_2,0.000182,0.360981,0.001892,0.007364,0.000000,0.000000,0.0,0.000284,0.006090,...,0.246175,0.542221,0.402793,0.335923,0.241655,0.232638,0.137530,0.278195,0.306376,0.078349
TCGA-OR-A5J2,ACC_2,0.000662,0.028076,0.016630,0.094906,0.001782,0.002093,0.0,0.052871,0.002012,...,0.057344,0.810534,0.198127,0.339276,1.000000,0.255464,0.184041,0.413199,0.303635,0.039638
TCGA-OR-A5J3,ACC_3,0.000675,0.504978,0.004538,0.036593,0.002623,0.001027,0.0,0.002378,0.000851,...,0.457210,0.488562,0.499732,0.410338,0.493998,0.576924,0.231120,0.519779,0.540690,0.028952
TCGA-OR-A5J5,ACC_3,0.039602,0.519118,0.005965,0.081847,0.000000,0.001342,0.0,0.000142,0.001235,...,0.145073,0.509690,0.383014,0.394384,0.254431,0.305898,0.222402,0.371465,0.304460,0.073477
TCGA-OR-A5J6,ACC_1,0.000141,0.017320,0.001465,0.003721,0.000000,0.000000,0.0,0.007885,0.002986,...,0.083668,0.411400,0.346296,0.393668,0.321142,0.231314,0.780143,0.318338,0.992842,0.742167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-OU-A5PI,ACC_3,0.001371,0.008835,0.066020,0.021505,0.000000,0.000000,0.0,0.000697,0.002046,...,0.342442,0.369873,0.377868,0.418249,0.551750,0.355757,0.124816,0.525230,0.515990,0.025401
TCGA-PA-A5YG,ACC_1,0.063596,0.007164,0.010554,0.031371,0.001023,0.002404,0.0,0.005909,0.006182,...,0.041263,0.615558,0.462613,0.325033,0.593977,0.227852,0.408315,0.638969,0.301985,0.038206
TCGA-PK-A5H9,ACC_1,0.048619,0.009463,0.004498,0.020371,0.000000,0.000605,0.0,0.001266,0.311465,...,0.045505,0.910912,0.386055,0.313144,0.436139,0.292534,0.428219,0.188895,0.312588,0.134155
TCGA-PK-A5HA,ACC_1,0.000349,0.012814,0.076510,0.023222,0.003660,0.000000,0.0,0.001905,0.004733,...,0.065540,0.652931,0.471015,0.265587,0.358783,0.295931,0.187683,0.639133,0.910880,0.228176


In [48]:
# miR self, self-decoded_miR is target out
# miR_paths = sorted(glob.glob('data_miR_col_norm/*.tsv'))
# for miR_path in miR_paths:
# for miR_path in [miR_paths[2]]:
paths = sorted(glob.glob('data_GEXP_5k_col_norm/*.tsv'))
for path in paths:    
    
    frame = pd.read_csv(path, sep = '\t', index_col = 0)
    c = frame.index.name
    print(c)
    features = frame.columns[1:] # custom data type
    
    original_dim = len(features)
    feature_dim = len(features)
    latent_dim = 100
    
    encoder_inputs = keras.Input(shape=(feature_dim,))
    z_mean_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_1")(encoder_inputs)
    z_mean_dense_batchnorm = layers.BatchNormalization()(z_mean_dense_linear)
    z_mean_encoded = layers.Activation('relu')(z_mean_dense_batchnorm)

    z_log_var_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_2")(encoder_inputs)
    z_log_var_dense_batchnorm = layers.BatchNormalization()(z_log_var_dense_linear)
    z_log_var_encoded = layers.Activation('relu')(z_log_var_dense_batchnorm)

    latent_space = layers.Lambda(compute_latent, output_shape=(latent_dim,), name="latent_space")([z_mean_encoded, z_log_var_encoded])

    decoder_to_reconstruct = layers.Dense(feature_dim, kernel_initializer='glorot_uniform', activation='sigmoid')
    decoder_outputs = decoder_to_reconstruct(latent_space)

    learning_rate = 0.0005
    kappa = 1

    beta = K.variable(0)

    adam = optimizers.Adam(learning_rate=learning_rate)
    vae_layer = CustomVariationalLayer()([encoder_inputs, decoder_outputs])
    vae = Model(encoder_inputs, vae_layer)
    vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

    epochs=100

    fit_start = time.time()
    history = vae.fit(frame.iloc[:, 1:],  
                epochs=epochs, batch_size=50, shuffle=True,
                callbacks=[WarmUpCallback(beta, kappa)],
                     # verbose=0
                     )
    fit_end = time.time() - fit_start

    plt.plot(history.history['loss'],label="loss")    

    plt.title(
        c+' loss for self-decode'+
        '\nTybalt VAE train\n'
             )
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.annotate(
        # 'Feature set = '+'miR n v = 743'+
        'Feature set = '+'GEXP MAD n = 5k'+
                 '\nLatent dim = '+str(latent_dim)+
                 '\nLayer type = dense'+
                 '\nNormalization = divide by max()',
                xy=(.4, .8), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                )

    plt.legend(loc="lower left")
    # break
#   2022-11-14, loss plot miR template

    # plt.savefig(
    #     'train-on-one_loss_plots/'+c+'_trained_for_n=24_'+
    #     '_predictions_pltv'+str(plt_v)+'.png', bbox_inches='tight')
    
    plt.savefig(
        'self_common_MAD_loss_plots/'+c+'_self_'+
        'v'+str(plt_v)+'.png', bbox_inches='tight')
    
    plt.close()

    # Decode self
    
    encoder = Model(encoder_inputs, z_mean_encoded)
    decoder_input = keras.Input(shape=(latent_dim, ))
    _x_decoded_mean = decoder_to_reconstruct(decoder_input)
    decoder = Model(decoder_input, _x_decoded_mean)
    
    decoded = pd.DataFrame(
        decoder.predict(encoder.predict(
            frame.iloc[:, 1:])),
            index=frame.index, columns=frame.iloc[:, 1:].columns)

    y_df = frame.Labels
    decoded_labeled = pd.concat( [pd.DataFrame(y_df), decoded] , axis =1)
        
# 2022-11-14, miR self-decoded template out        
    decoded_labeled.to_csv(
        # 'train-on-one_decoded/'+c+'_trained_'+c_not+'_decoded_'+
        'self-decoded_GEXP/'+c+'_GEXP_self_decoded_'+
        str(round(fit_end,2))+'_fit_time.tsv',
        sep = '\t')

    print(str(c) + ' done')

print('decode experiment done')

ACC


2022-11-14 16:08:14.372397: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-14 16:08:14.510086: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-14 16:08:14.564349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Train on 76 samples
Epoch 1/100


2022-11-14 16:08:14.810222: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-14 16:08:14.956247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-14 16:08:15.005618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


76/76 [==============================] - 1s 13ms/sample - loss: 3520.1821
Epoch 2/100
76/76 [==============================] - 0s 524us/sample - loss: 3478.6281
Epoch 3/100
76/76 [==============================] - 0s 502us/sample - loss: 3440.0716
Epoch 4/100
76/76 [==============================] - 0s 566us/sample - loss: 3402.6436
Epoch 5/100
76/76 [==============================] - 0s 589us/sample - loss: 3359.7168
Epoch 6/100


...

74/74 [==============================] - 2s 30ms/sample - loss: 3517.4586
Epoch 2/100
74/74 [==============================] - 0s 1ms/sample - loss: 3483.3669
Epoch 3/100
50/74 [===================>..........] - ETA: 0s - loss: 3461.8193

2022-11-14 16:13:50.025775: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


74/74 [==============================] - 0s 1ms/sample - loss: 3453.9016
Epoch 4/100
74/74 [==============================] - 0s 605us/sample - loss: 3418.8553
Epoch 5/100
74/74 [==============================] - 0s 696us/sample - loss: 3385.6891
Epoch 6/100
74/74 [==============================] - 0s 553us/sample - loss: 3355.2407
Epoch 7/100
74/74 [==============================] - 0s 479us/sample - loss: 3323.4760
Epoch 8/100
74/74 [==============================] - 0s 807us/sample - loss: 3292.5013
Epoch 9/100
74/74 [==============================] - 0s 420us/sample - loss: 3263.4566
Epoch 10/100
74/74 [==============================] - 0s 609us/sample - loss: 3239.0268
Epoch 11/100
74/74 [==============================] - 0s 789us/sample - loss: 3207.8244
Epoch 12/100
74/74 [==============================] - 0s 391us/sample - loss: 3181.4756
Epoch 13/100
74/74 [==============================] - 0s 585us/sample - loss: 3156.9875
Epoch 14/100
74/74 [==============================] -

/Users/karlberb/miniforge3/envs/tf_M1_py3dot9/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-11-14 16:13:53.640580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-14 16:13:54.188680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


UVM done
decode experiment done


In [49]:
pwd

'/Users/karlberb/Documents/work/sample_count/synthetic_TCGA_data_gen/g_devel'

In [36]:
frame

,Labels,N:GEXP::?:100133144:,N:GEXP::?:100134869:,N:GEXP::?:10357:,N:GEXP::?:10431:,N:GEXP::?:155060:,N:GEXP::?:388795:,N:GEXP::?:390284:,N:GEXP::?:57714:,N:GEXP::?:645851:,...,N:GEXP::ZWINT:11130:,N:GEXP::ZXDA:7789:,N:GEXP::ZXDB:158586:,N:GEXP::ZXDC:79364:,N:GEXP::ZYG11A:440590:,N:GEXP::ZYG11B:79699:,N:GEXP::ZYX:7791:,N:GEXP::ZZEF1:23140:,N:GEXP::ZZZ3:26009:,N:GEXP::psiTPTE22:387590:
TCGA-OR-A5J1,ACC_2,0.250891,0.172537,0.895407,0.654514,0.432039,0.000000,0.499653,0.356096,0.037854,...,0.065120,0.211854,0.446666,0.352663,0.002898,0.401766,0.168513,0.422368,0.598797,0.449648
TCGA-OR-A5J2,ACC_2,0.205984,0.394042,0.486809,0.419879,0.313967,0.039988,0.226883,0.665553,0.063465,...,0.194275,0.586468,0.354482,1.000000,0.189461,0.722474,0.280092,0.691845,0.805320,0.365544
TCGA-OR-A5J3,ACC_3,0.132901,0.287598,0.519292,0.339372,0.548836,0.058850,0.333879,0.204703,0.264626,...,0.084270,0.407213,0.629294,0.307000,0.003574,0.499659,0.243076,0.540960,0.439277,0.079668
TCGA-OR-A5J5,ACC_3,0.000000,0.067867,0.323699,0.756452,0.691878,0.076937,0.268604,0.066413,0.081398,...,0.372382,0.182753,0.261764,0.298929,0.070095,0.343163,0.216741,0.416055,0.515134,0.537290
TCGA-OR-A5J6,ACC_1,0.000000,0.195860,0.401717,0.404788,0.235074,0.277543,0.460251,0.180393,0.044046,...,0.027981,0.131853,0.149812,0.300255,0.047201,0.493097,0.064812,0.290593,0.477773,0.181182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-OU-A5PI,ACC_3,0.102481,0.191317,0.685500,0.664650,0.163458,0.056625,0.148279,0.426995,0.164755,...,0.142474,0.655000,0.624365,0.526897,0.003439,0.548609,0.253831,0.480569,0.550405,0.191285
TCGA-PA-A5YG,ACC_1,0.416224,0.248710,0.543014,0.250959,0.104149,0.091836,0.400798,0.163800,0.048583,...,0.052851,0.697129,0.542725,0.208826,0.000000,0.857116,0.269858,0.270593,0.516460,0.003486
TCGA-PK-A5H9,ACC_1,0.944846,0.760503,0.404659,0.268016,0.290794,0.000000,0.121006,0.392460,0.165010,...,0.031387,0.658621,0.328946,0.343233,0.025262,1.000000,0.130224,0.585331,0.790141,0.008112
TCGA-PK-A5HA,ACC_1,0.000000,0.144899,0.378193,0.283989,0.329138,0.273768,0.095583,0.222431,0.014483,...,0.127746,0.633336,0.479050,0.415517,0.000000,0.561547,0.266426,0.240179,0.327848,0.233451


In [16]:
miR_path

'data_miR_col_norm/ACCmir_col_norm_743.tsv'

In [15]:
ls data_miR_col_norm

ACCmir_col_norm_743.tsv       OVmir_col_norm_743.tsv
BLCAmir_col_norm_743.tsv      PAADmir_col_norm_743.tsv
BRCAmir_col_norm_743.tsv      PCPGmir_col_norm_743.tsv
CESCmir_col_norm_743.tsv      PRADmir_col_norm_743.tsv
COADREADmir_col_norm_743.tsv  SARCmir_col_norm_743.tsv
ESCCmir_col_norm_743.tsv      SKCMmir_col_norm_743.tsv
GEAmir_col_norm_743.tsv       TGCTmir_col_norm_743.tsv
HNSCmir_col_norm_743.tsv      THCAmir_col_norm_743.tsv
KIRPmir_col_norm_743.tsv      THYMmir_col_norm_743.tsv
LUADmir_col_norm_743.tsv      UCECmir_col_norm_743.tsv
LUSCmir_col_norm_743.tsv      UVMmir_col_norm_743.tsv
MESOmir_col_norm_743.tsv


In [17]:
miR_frame = pd.read_csv(miR_path, sep = '\t', index_col = 0)

FileNotFoundError: [Errno 2] No such file or directory: 'data_miR_col_norm/ACCmir_col_norm_743.tsv'

In [69]:
df.head()  # DELETE (demo vers_)

,Labels,N:GEXP::?:100133144:,N:GEXP::?:100134869:,N:GEXP::?:10357:,N:GEXP::?:10431:,N:GEXP::?:155060:,N:GEXP::?:388795:,N:GEXP::?:390284:,N:GEXP::?:57714:,N:GEXP::?:645851:,...,N:GEXP::ZWINT:11130:,N:GEXP::ZXDA:7789:,N:GEXP::ZXDB:158586:,N:GEXP::ZXDC:79364:,N:GEXP::ZYG11A:440590:,N:GEXP::ZYG11B:79699:,N:GEXP::ZYX:7791:,N:GEXP::ZZEF1:23140:,N:GEXP::ZZZ3:26009:,N:GEXP::psiTPTE22:387590:
ACC,,,,,,,,,,,,,,,,,,,,,
TCGA-OR-A5J1,ACC_2,3.2661,3.9385,149.130,2034.1,274.25,0.0000,11.5270,282.420,1.4409,...,146.970,20.653,351.58,1050.90,0.4803,648.41,1841.00,1157.50,596.06,715.66
TCGA-OR-A5J2,ACC_2,2.6815,8.9948,81.078,1304.9,199.30,0.4026,5.2342,527.850,2.4158,...,438.460,57.173,279.02,2979.90,31.4050,1166.00,3060.00,1896.00,801.64,581.80
TCGA-OR-A5J3,ACC_3,1.7301,6.5650,86.488,1054.7,348.39,0.5925,7.7026,162.350,10.0730,...,190.190,39.698,495.33,914.83,0.5925,806.40,2655.60,1482.50,437.27,126.80
TCGA-OR-A5J5,ACC_3,0.0000,1.5492,53.912,2350.9,439.19,0.7746,6.1967,52.672,3.0984,...,840.430,17.816,206.04,890.78,11.6190,553.83,2367.90,1140.20,512.78,855.15
TCGA-OR-A5J6,ACC_1,0.0000,4.4709,66.906,1258.0,149.22,2.7943,10.6180,143.070,1.6766,...,63.151,12.854,117.92,894.73,7.8240,795.81,708.07,796.37,475.59,288.37


In [ ]:
# Get from the primary subset object
brca_df = pd.read_csv('data_5k/BRCA_mad_5k.tsv', sep="\t", index_col=0) # Delete

In [20]:
stacked.Labels.str.split('_').str[0].unique()

array(['ACC', 'BLCA', 'BRCA', 'CESC', 'COADREAD', 'ESCC', 'GEA', 'HNSC',
       'KIRCKICH', 'KIRP', 'LIHCCHOL', 'LUAD', 'LUSC', 'MESO', 'OV',
       'PAAD', 'PCPG', 'PRAD', 'SARC', 'SKCM', 'TGCT', 'THCA', 'THYM',
       'UCEC', 'UVM'], dtype=object)

In [23]:
stacked.columns[1:]

Index(['N:GEXP::ALB:213:', 'N:GEXP::TG:7038:', 'N:GEXP::COL1A1:1277:',
       'N:GEXP::FN1:2335:', 'N:GEXP::KRT5:3852:', 'N:GEXP::KRT14:3861:',
       'N:GEXP::KRT6A:3853:', 'N:GEXP::ADAM6:8755:', 'N:GEXP::SERPINA1:5265:',
       'N:GEXP::COL3A1:1281:',
       ...
       'N:GEXP::C14orf1:11161:', 'N:GEXP::SMCHD1:23347:',
       'N:GEXP::ARL2BP:23568:', 'N:GEXP::PITPNB:23760:', 'N:GEXP::KDM5A:5927:',
       'N:GEXP::CNIH:10175:', 'N:GEXP::ACADS:35:', 'N:GEXP::YIPF2:78992:',
       'N:GEXP::MEPCE:56257:', 'N:GEXP::PPARG:5468:'],
      dtype='object', length=5000)

In [38]:
c

'ACC'

In [33]:
print(list(stacked.Labels.str.split('_').str[0].unique()).remove(c))

None


In [36]:
primary_list = list(stacked.Labels.str.split('_').str[0].unique())

In [40]:
primary_list.remove(c) # key to the step-up experiment

In [41]:
primary_list

['BLCA',
 'BRCA',
 'CESC',
 'COADREAD',
 'ESCC',
 'GEA',
 'HNSC',
 'KIRCKICH',
 'KIRP',
 'LIHCCHOL',
 'LUAD',
 'LUSC',
 'MESO',
 'OV',
 'PAAD',
 'PCPG',
 'PRAD',
 'SARC',
 'SKCM',
 'TGCT',
 'THCA',
 'THYM',
 'UCEC',
 'UVM']

In [ ]:
# 2022-11-14, working on miR experiments, can I run all four simultaneously?

# template: (b_train-on-one.ipynb) # This is the latest train-on-one, predict-on-another

In [49]:
# this looks important; step through. Is this the train-on-one loop?

# train_paths_5k = sorted(glob.glob('data_5k/*.tsv'))

# for train_path in train_paths_5k:
for c in stacked.Labels.str.split('_').str[0].unique():
    print(c)
    primary_subset = stacked[stacked.Labels.str.contains(c)]
    # train_df = pd.read_csv(train_path, sep="\t", index_col=0)
    
    # break # careful, there is file-writing right in the loop
    # train_cohort = train_df.index.name # lost the index name when stacking, now its c var
    # features = train_df.columns[1:]
    features = stacked.columns[1:]
    
    original_dim = len(features)
    feature_dim = len(features)
    latent_dim = 100
    
    encoder_inputs = keras.Input(shape=(feature_dim,))
    z_mean_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_1")(encoder_inputs)
    z_mean_dense_batchnorm = layers.BatchNormalization()(z_mean_dense_linear)
    z_mean_encoded = layers.Activation('relu')(z_mean_dense_batchnorm)

    z_log_var_dense_linear = layers.Dense(latent_dim, kernel_initializer='glorot_uniform', name="encoder_2")(encoder_inputs)
    z_log_var_dense_batchnorm = layers.BatchNormalization()(z_log_var_dense_linear)
    z_log_var_encoded = layers.Activation('relu')(z_log_var_dense_batchnorm)

    latent_space = layers.Lambda(compute_latent, output_shape=(latent_dim,), name="latent_space")([z_mean_encoded, z_log_var_encoded])

    decoder_to_reconstruct = layers.Dense(feature_dim, kernel_initializer='glorot_uniform', activation='sigmoid')
    decoder_outputs = decoder_to_reconstruct(latent_space)

    learning_rate = 0.0005
    kappa = 1

    beta = K.variable(0)

    adam = optimizers.Adam(learning_rate=learning_rate)
    vae_layer = CustomVariationalLayer()([encoder_inputs, decoder_outputs])
    vae = Model(encoder_inputs, vae_layer)
    vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

    epochs=100

    fit_start = time.time()
    history = vae.fit(primary_subset.iloc[:, 1:],  
                epochs=epochs, batch_size=50, shuffle=True,
                callbacks=[WarmUpCallback(beta, kappa)],
                     verbose=0)
    fit_end = time.time() - fit_start

    plt.plot(history.history['loss'],label="loss")

    plt.title(
        c+' loss for direct predict on other cohorts n = 24'+
        '\nTybalt VAE train\n'
             )
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.annotate('Feature set = '+'common MAD 5000k'+
                 '\nLatent dim = '+str(latent_dim)+
                 '\nLayer type = dense'+
                 '\nNormalization = divide by max()',
                xy=(.4, .8), xycoords='figure fraction',
                horizontalalignment='left', verticalalignment='top',
                )

    plt.legend(loc="lower left")
    plt.savefig(
        'train-on-one_loss_plots/'+c+'_trained_for_n=24_'+
        '_predictions_pltv'+str(plt_v)+'.png', bbox_inches='tight')
    plt.close()
    # break # model reset checkpoint, code block to b_devel, plots to 2022-11-10_no_reset_test and to slide deck 4
    
    # predict_paths_5k = sorted(glob.glob('data_5k/*.tsv')) # here is the start of the hold-out section
    
    # predict_paths_5k.remove(train_path) # Overwrite predict paths with train path dropped. Tricky, replace
    primary_list = list(stacked.Labels.str.split('_').str[0].unique())
    primary_list.remove(c)
    for c_not in primary_list:
        
        # predict_df = pd.read_csv(j, sep="\t", index_col=0)
        # predict_cohort = predict_df.index.name # this is the nth predict cohort
        primary_not_subset = stacked[stacked.Labels.str.contains(c_not)]
        
        encoder = Model(encoder_inputs, z_mean_encoded)
        decoder_input = keras.Input(shape=(latent_dim, ))
        _x_decoded_mean = decoder_to_reconstruct(decoder_input)
        decoder = Model(decoder_input, _x_decoded_mean)

        # test = predict_df.iloc[:, 1:]
        y_df = primary_not_subset.Labels
        
        # print(decoder.predict(encoder.predict(predict_df.iloc[:, 1:])))
        # print(test.index)
        # print(test.columns)
        testOut = pd.DataFrame(decoder.predict(encoder.predict(primary_not_subset.iloc[:, 1:])),
                               index=primary_not_subset.index, columns=primary_not_subset.iloc[:, 1:].columns)
        # not writing the latent spaces, add this in a future refinement, probably back-port from train-on_all
        # which will be ripped out from b_step_up
        testOut_labeled = pd.concat( [pd.DataFrame(y_df), testOut] , axis =1)
        testOut_labeled.to_csv(
            'train-on-one_decoded/'+c+'_trained_'+c_not+'_decoded_'+
            str(round(fit_end,2))+'_fit_time.tsv',
            sep = '\t')

        print(str(c) + ' done')
        # break
    # break
print('Train-on-one experiment done')

ACC


2022-11-10 15:27:01.040842: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-10 15:27:01.155228: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-10 15:27:01.204711: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-10 15:27:01.438141: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-10 15:27:01.562906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-10 15:27:01.605377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-10 15:27:02.140224: I tensorflow/core/grappler/optimizers/cust

ACC done


/Users/karlberb/miniforge3/envs/tf_M1_py3dot9/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-11-10 15:27:06.029679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-10 15:27:06.269688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


ACC done


...

UVM done


/Users/karlberb/miniforge3/envs/tf_M1_py3dot9/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-11-10 15:46:57.877636: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-10 15:46:58.476325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


UVM done


/Users/karlberb/miniforge3/envs/tf_M1_py3dot9/lib/python3.9/site-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-11-10 15:46:59.302681: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-10 15:46:59.936647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


UVM done
Train-on-one experiment done


In [47]:
check_decoded = pd.read_csv(path, sep="\t", index_col=0)

In [48]:
check_decoded

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-2F-A9KO,BLCA_2,0.250637,0.258057,0.268317,0.257099,0.237514,0.265944,0.234808,0.251441,0.254603,...,0.391206,0.276373,0.435641,0.309255,0.359462,0.345481,0.310616,0.343847,0.389621,0.247402
TCGA-2F-A9KP,BLCA_2,0.194274,0.219538,0.232924,0.216289,0.206727,0.237237,0.203608,0.220831,0.222865,...,0.399941,0.257975,0.414973,0.305211,0.321591,0.300369,0.279032,0.330401,0.379641,0.216285
TCGA-2F-A9KQ,BLCA_2,0.271836,0.290882,0.292923,0.283768,0.274932,0.291748,0.279892,0.286083,0.287526,...,0.392897,0.311263,0.451493,0.337189,0.371807,0.354897,0.350414,0.375361,0.397817,0.285833
TCGA-2F-A9KR,BLCA_1,0.263763,0.286219,0.286180,0.284906,0.269792,0.282094,0.262673,0.284718,0.285614,...,0.397603,0.316108,0.440257,0.343644,0.373510,0.356079,0.342179,0.372055,0.417940,0.282861
TCGA-2F-A9KT,BLCA_1,0.228537,0.261161,0.241728,0.237686,0.241560,0.266547,0.224930,0.233142,0.244609,...,0.392600,0.276578,0.421402,0.303617,0.328425,0.324495,0.294744,0.352128,0.408783,0.241157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA56,BLCA_1,0.238115,0.261892,0.262777,0.249768,0.242043,0.264090,0.234637,0.255230,0.251610,...,0.408945,0.287983,0.414952,0.325195,0.343072,0.340931,0.317783,0.353381,0.387536,0.245686
TCGA-ZF-AA58,BLCA_1,0.210559,0.229422,0.229753,0.222328,0.200068,0.232017,0.201545,0.208206,0.207913,...,0.378071,0.244429,0.419155,0.301361,0.331318,0.310866,0.286212,0.328638,0.378077,0.204849
TCGA-ZF-AA5H,BLCA_1,0.189527,0.202081,0.207959,0.202062,0.188045,0.226639,0.184971,0.189261,0.213654,...,0.409961,0.236510,0.419178,0.289519,0.309319,0.304097,0.281105,0.325800,0.373702,0.198847
TCGA-ZF-AA5N,BLCA_2,0.225260,0.254950,0.247940,0.241407,0.230776,0.260822,0.236963,0.235661,0.246769,...,0.405002,0.286186,0.426293,0.329150,0.340972,0.331351,0.320058,0.365965,0.392399,0.236632


### Original notes, pd.read_csv('

In [325]:
blnk_frm.to_csv(
            't1pa_F1s/'+chrt+'_predict_on_n=24.tsv',
            sep = '\t')

In [338]:
bfr = pd.read_csv('t1pa_F1s/'+chrt+'_predict_on_n=24.tsv', sep = '\t', index_col = 0)

In [339]:
bfr

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
BLCA_trained_ACC_predict_rndm_frst_clf,0.502039,0.117223,0.614796,0.146019,0.529435,0.054791,0.536245,0.082535,0.576001,0.093903
BLCA_trained_BRCA_predict_rndm_frst_clf,0.506716,0.029684,0.520079,0.012900,0.527048,0.030649,0.529623,0.033118,0.535627,0.028430
BLCA_trained_CESC_predict_rndm_frst_clf,0.534484,0.098818,0.522041,0.073374,0.527539,0.078088,0.497163,0.064038,0.481349,0.057453
BLCA_trained_COADREAD_predict_rndm_frst_clf,0.320525,0.065152,0.355617,0.049271,0.340839,0.053788,0.320924,0.044449,0.331923,0.031090
BLCA_trained_ESCC_predict_rndm_frst_clf,0.596318,0.059760,0.610243,0.047932,0.603200,0.069953,0.614632,0.145403,0.552390,0.056017
BLCA_trained_GEA_predict_rndm_frst_clf,0.251844,0.056349,0.264135,0.026813,0.226648,0.038634,0.278596,0.052243,0.248634,0.020387
BLCA_trained_HNSC_predict_rndm_frst_clf,0.459626,0.036710,0.429202,0.052487,0.454902,0.066725,0.448692,0.050250,0.443373,0.047134
BLCA_trained_KIRCKICH_predict_rndm_frst_clf,0.971970,0.007615,0.970002,0.011615,0.965825,0.009415,0.972449,0.011959,0.973607,0.009605
BLCA_trained_KIRP_predict_rndm_frst_clf,0.597778,0.106181,0.607500,0.064280,0.557377,0.053336,0.562136,0.058894,0.549304,0.061581
BLCA_trained_LIHCCHOL_predict_rndm_frst_clf,0.506512,0.085734,0.461435,0.061837,0.452226,0.070450,0.494188,0.099690,0.452261,0.035855


#### Plot original - send to d_, looks like there is already three options
d_F1_templates made

In [38]:
pre_train_paths = ['data/ACC.tsv', # DELETE (demo vers_
 'data/BLCA.tsv',
 # 'data/BRCA.tsv',
 'data/CESC.tsv',
 'data/COADREAD.tsv',
 'data/ESCC.tsv',
 'data/GEA.tsv',
 'data/HNSC.tsv',
 'data/KIRCKICH.tsv',
 'data/KIRP.tsv',
 'data/LIHCCHOL.tsv',
 'data/LUAD.tsv',
 'data/LUSC.tsv',
 'data/MESO.tsv',
 'data/OV.tsv',
 'data/PAAD.tsv',
 'data/PCPG.tsv',
 'data/PRAD.tsv',
 'data/SARC.tsv',
 'data/SKCM.tsv',
 'data/TGCT.tsv',
 'data/THCA.tsv',
 'data/THYM.tsv',
 'data/UCEC.tsv',
 'data/UVM.tsv']

In [57]:
pre_train_paths # DELETE (demo vers_

['data/ACC.tsv',
 'data/BLCA.tsv',
 'data/CESC.tsv',
 'data/COADREAD.tsv',
 'data/ESCC.tsv',
 'data/GEA.tsv',
 'data/HNSC.tsv',
 'data/KIRCKICH.tsv',
 'data/KIRP.tsv',
 'data/LIHCCHOL.tsv',
 'data/LUAD.tsv',
 'data/LUSC.tsv',
 'data/MESO.tsv',
 'data/OV.tsv',
 'data/PAAD.tsv',
 'data/PCPG.tsv',
 'data/PRAD.tsv',
 'data/SARC.tsv',
 'data/SKCM.tsv',
 'data/TGCT.tsv',
 'data/THCA.tsv',
 'data/THYM.tsv',
 'data/UCEC.tsv',
 'data/UVM.tsv']

In [40]:
len(pre_train_paths) # DELETE (demo vers_

24